In [1]:
#import dependencies
import geemap
import ee

In [3]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

In [12]:
# Define the Area of Interest (AOI)
aoi = ee.Geometry.Polygon(
    [[[-3.8225909618719167, 40.53241723973516],
      [-3.8225909618719167, 40.233239070237204],
      [-3.4737750439031667, 40.233239070237204],
      [-3.4737750439031667, 40.53241723973516]]])

In [13]:
# Define start and end dates
start_date = '2022-05-01'
end_date = '2022-12-31'

In [14]:
# Applies scaling factors.
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True) \
        .addBands(thermal_bands, None, True)

In [15]:
# Cloud mask
def mask_l8sr(col):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloud_shadow_bit_mask = (1 << 3)
    clouds_bit_mask = (1 << 5)
    # Get the pixel QA band.
    qa = col.select('QA_PIXEL')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0) \
        .And(qa.bitwiseAnd(clouds_bit_mask).eq(0))
    return col.updateMask(mask)

In [16]:
# Filter the collection, first by the AOI, and then by date.
image_collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                    .filterDate(start_date, end_date)
                    .filterBounds(aoi)
                    .map(apply_scale_factors)
                    .map(mask_l8sr))

image = image_collection.median()

visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

Map = geemap.Map()
Map.addLayer(image, visualization, 'True Color (432)', False)

In [17]:
# NDVI
ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
Map.addLayer(ndvi, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'ndvi', False)

In [19]:
# NDVI statistics
ndvi_min = ee.Number(ndvi.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).values().get(0))

ndvi_max = ee.Number(ndvi.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).values().get(0))


In [20]:
# Fraction of vegetation
fv = (ndvi.subtract(ndvi_min).divide(ndvi_max.subtract(ndvi_min))).pow(ee.Number(2)) \
    .rename('FV')

em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')

thermal = image.select('ST_B10').rename('thermal')

lst = thermal.expression(
    '(tb / (1 + (0.00115 * (tb/0.48359547432)) * log(em))) - 273.15',
    {'tb': thermal.select('thermal'),
     'em': em}).rename('LST')

lst_vis = {
    'min': 25,
    'max': 50,
    'palette': [
        '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
        '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
        '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
        'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
        'ff0000', 'de0101', 'c21301', 'a71001', '911003']
}

Map.addLayer(lst, lst_vis, 'LST AOI')
Map.centerObject(aoi, 10)

In [22]:
# Urban Heat Island

# 1. Normalized UHI

# Calculate mean and standard deviation of LST in the AOI
lst_stats = lst.reduceRegion(
    reducer=ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs=True),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Extract mean and standard deviation values
lst_mean = ee.Number(lst_stats.get('LST_mean'))
lst_std = ee.Number(lst_stats.get('LST_stdDev'))

print('Mean LST in AOI', lst_mean)
print('STD LST in AOI', lst_std)

# Calculate UHI
uhi = lst.subtract(lst_mean).divide(lst_std).rename('UHI')

# Visualization parameters for UHI
uhi_vis = {
    'min': -4,
    'max': 4,
    'palette': ['313695', '74add1', 'fed976', 'feb24c', 'fd8d3c', 'fc4e2a', 'e31a1c', 'b10026']
}

# Add UHI layer to the map
Map.addLayer(uhi, uhi_vis, 'UHI AOI')


Mean LST in AOI ee.Number({
  "functionInvocationValue": {
    "functionName": "Dictionary.get",
    "arguments": {
      "dictionary": {
        "functionInvocationValue": {
          "functionName": "Image.reduceRegion",
          "arguments": {
            "geometry": {
              "functionInvocationValue": {
                "functionName": "GeometryConstructors.Polygon",
                "arguments": {
                  "coordinates": {
                    "constantValue": [
                      [
                        [
                          -3.8225909618719167,
                          40.53241723973516
                        ],
                        [
                          -3.8225909618719167,
                          40.233239070237204
                        ],
                        [
                          -3.4737750439031667,
                          40.233239070237204
                        ],
                        [
                          -3.4

In [23]:
# Urban Thermal Field variance Index (UTFVI)

utfvi = lst.subtract(lst_mean).divide(lst).rename('UTFVI')
utfvi_vis = {
    'min': -1,
    'max': 0.3,
    'palette': ['313695', '74add1', 'fed976', 'feb24c', 'fd8d3c', 'fc4e2a', 'e31a1c',
                'b10026']
}
Map.addLayer(utfvi, utfvi_vis, 'UTFVI AOI')


In [29]:
#displaying the heat distribution of map
Map

Map(bottom=49637.0, center=[40.594403029572845, -3.563345308252553], controls=(WidgetControl(options=['positio…